# Bibliotecas iniciales

WordNetLemmatizer permite ubicar palabras clave sin necesidad de buscar las palabras tal cual dentro de un texto. Por ejemplo, "work", "working", "worked" se reconocen como una misma palabras (siendo todas variaciones de una sola raíz).

In [13]:
import random # Selección de respuesta aleatoria.
import json # Corpus.
import pickle # Serialización.
import numpy as np

import nltk # Natural Language Toolkit.
from nltk.stem import WordNetLemmatizer # Lematización.
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

from tensorflow.keras.models import Sequential # Modelo de red neuronal.
from tensorflow.keras.layers import Dense, Activation, Dropout # Capas de la red.
from tensorflow.keras.optimizers import SGD # Descenso de gradiente estocástico como optimizador de función de costo.

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Leona\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Leona\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Leona\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# Iniciación del Lemmatizer

In [14]:
lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents.json').read()) # Carga del corpus.

words = [] # Lista de palabras.
classes = [] # Lista de clases.
documents = [] # Lista de documentos.
ignore_words = ['?', '!', '¡', '.', ',', ';'] # Palabras ignoradas.

# Consideramos el .json como un diccionario de Python y accedemos a sus claves y valores por medio de un ciclo for.
# El diccionario como tal es "intents", y en cada clave del diccionario se encuentra un subdiccionario que engloba cada tipo de respuesta del chatbot.
# Cada subdiccionario se engloba con la llave "patterns" y su valor es una lista de patrones de respuesta "responses"

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern) # Tokenización de patrones de respuesta. Ejemplo: ['hi', 'how', 'are', 'you'] es una tokenización de "hi how are you".
        words.extend(word_list) # Toma las palabras y las añade a lista.
        documents.append((word_list, intent['tag'])) # Añadimos las palabras a la lista de documentos categorizados por su tag correspondiente.
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
            
print(documents)


[(['hola'], 'greetings'), (['buenos', 'dias'], 'greetings'), (['buenas', 'tardes'], 'greetings'), (['que', 'tal'], 'greetings'), (['mucho', 'gusto'], 'greetings'), (['hey'], 'greetings'), (['gracias'], 'goodbye'), (['hasta', 'luego'], 'goodbye'), (['saludos'], 'goodbye'), (['muchas', 'gracias'], 'goodbye'), (['adios'], 'goodbye'), (['le', 'agradezco'], 'goodbye'), (['ubicacion'], 'shop'), (['donde', 'estan', 'ubicados'], 'shop'), (['local'], 'shop'), (['donde', 'los', 'puedo', 'encontrar'], 'shop'), (['establecimiento'], 'shop'), (['contacto'], 'shop'), (['a', 'que', 'hora', 'abren'], 'hours'), (['horario'], 'hours'), (['hora'], 'hours'), (['horas', 'de', 'apertura'], 'hours'), (['abierto'], 'hours'), (['que', 'venden'], 'stocks'), (['productos'], 'stocks'), (['categorias'], 'stocks'), (['oferta'], 'stocks'), (['venta'], 'stocks'), (['stock'], 'stocks'), (['inventario'], 'stocks'), (['cpu', 'para', 'videojuegos'], 'CPU-Gaming'), (['procesador', 'gaming'], 'CPU-Gaming'), (['procesador',

In [15]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_words] # Lematización de palabras.
words = sorted(set(words))

classes = sorted(set(classes))
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

print(words) # Imprimimos todas las palabras presentes en el json.

['8gb', 'RT', 'a', 'abierto', 'abren', 'adios', 'agradezco', 'aguanta', 'alimentar', 'alto', 'amd', 'apertura', 'argb', 'blender', 'buenas', 'buenos', 'categorias', 'computo', 'contacto', 'cpu', 'ddr3', 'ddr4', 'ddr5', 'de', 'deep', 'dia', 'donde', 'edicion', 'efectos', 'encontrar', 'establecimiento', 'estacion', 'estan', 'fsr', 'fuente', 'gaming', 'gpu', 'gracias', 'grafica', 'graficas', 'gtx', 'gusto', 'hasta', 'hey', 'highend', 'hola', 'hora', 'horario', 'horas', 'ia', 'inventario', 'juegos', 'le', 'learning', 'levanta', 'local', 'los', 'luego', 'machine', 'madre', 'maya', 'memoria', 'mother', 'motherboard', 'muchas', 'mucho', 'nvidia', 'oferta', 'para', 'performance', 'photoshop', 'placa', 'poder', 'potencia', 'procesador', 'procesamiento', 'produccion', 'productos', 'profesional', 'psu', 'puedo', 'que', 'ram', 'raytracing', 'render', 'rendimiento', 'rgb', 'rtx', 'rx', 'saludos', 'stick', 'stock', 'tal', 'tardes', 'tarjeta', 'trabajo', 'ubicacion', 'ubicados', 'venden', 'venta', 'v

# Modelo neuronal

Una red neuronal no trabaja con valores alfanuméricos, únicamente numéricos. Se requiere representar los números como valores numéricos con una técnica conocida como "bag of words". Esta técnica se representa con un arreglo donde, cuando se encuentra una ocurrencia de una palabra, su valor es 1 en el arreglo y en el resto de posiciones los valores son 0.

In [16]:
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
        
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])
    
random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5)) # Dropout de 50% de la neurona para prevenir overfitting.
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax')) # Tantas neuronas como clases. Softmax para que la salida sea una distribución probabilística entre 0 y 1.
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True) # Optimizador de función de costo.
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy']) # Función de costo y métrica de evaluación.

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1) # Entrenamiento del modelo.
model.save('chatbot_tienda.h5', hist) # Guardamos el modelo.
print("Modelo entrenado.")

C:\Users\Leona\AppData\Local\Temp\ipykernel_9352\4235938769.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


Epoch 1/200
19/19 [==============================] - 0s 1ms/step - loss: 2.4622 - accuracy: 0.1277
Epoch 2/200
19/19 [==============================] - 0s 1ms/step - loss: 2.4343 - accuracy: 0.1064
Epoch 3/200
19/19 [==============================] - 0s 997us/step - loss: 2.3926 - accuracy: 0.1702
Epoch 4/200
19/19 [==============================] - 0s 989us/step - loss: 2.3580 - accuracy: 0.1702
Epoch 5/200
19/19 [==============================] - 0s 944us/step - loss: 2.3381 - accuracy: 0.2128
Epoch 6/200
19/19 [==============================] - 0s 960us/step - loss: 2.2666 - accuracy: 0.2447
Epoch 7/200
19/19 [==============================] - 0s 989us/step - loss: 2.1740 - accuracy: 0.2872
Epoch 8/200
19/19 [==============================] - 0s 946us/step - loss: 2.2049 - accuracy: 0.2447
Epoch 9/200
19/19 [==============================] - 0s 943us/step - loss: 2.1291 - accuracy: 0.2872
Epoch 10/200
19/19 [==============================] - 0s 950us/step - loss: 2.1180 - accuracy: 